In [54]:
import json
import numpy as np
import pandas as pd
import random
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle

In [2]:
file = "/Users/meiyihe/Desktop/NYU/Fall2018/data/renttherunway_final_data.json"
df = pd.read_json(file, lines=True)

In [3]:
df.shape

(192544, 15)

In [4]:
df.head()

,age,body type,bust size,category,fit,height,item_id,rating,rented for,review_date,review_summary,review_text,size,user_id,weight
0,28.0,hourglass,34d,romper,fit,"5' 8""",2260466,10.0,vacation,"April 20, 2016",So many compliments!,An adorable romper! Belt and zipper were a lit...,14,420272,137lbs
1,36.0,straight & narrow,34b,gown,fit,"5' 6""",153475,10.0,other,"June 18, 2013",I felt so glamourous!!!,I rented this dress for a photo shoot. The the...,12,273551,132lbs
2,116.0,NaN,NaN,sheath,fit,"5' 4""",1063761,10.0,party,"December 14, 2015",It was a great time to celebrate the (almost) ...,This hugged in all the right places! It was a ...,4,360448,NaN
3,34.0,pear,34c,dress,fit,"5' 5""",126335,8.0,formal affair,"February 12, 2014",Dress arrived on time and in perfect condition.,I rented this for my company's black tie award...,8,909926,135lbs
4,27.0,athletic,34b,gown,fit,"5' 9""",616682,10.0,wedding,"September 26, 2016",Was in love with this dress !!!,I have always been petite in my upper body and...,12,151944,145lbs


In [19]:
#df.describe()
#df['category'].unique()
#df['rented for'].unique()

In [5]:
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)

In [6]:
len(df_train)

129004

In [7]:
# user: list of items
user_itemList = defaultdict(set)
# item: list of users
item_userList = defaultdict(set)

for idx, row in df.iterrows():
    user_itemList[ row['user_id'] ].add( row['item_id'] )

    item_userList[ row['item_id'] ].add( row['user_id'] )

In [33]:
user_list = []
item_list = []
for idx, row in df.iterrows():
    user_list.append(row['user_id'])
    item_list.append(row['item_id'])  

In [48]:
# build a non-visited user-item pair lists
non_visited = []
for idx, row in df.iterrows():
    curr_user = row['user_id']
    tmp_business = random.choice(item_list)
    while tmp_business in item_userList[curr_user]:
        tmp_business = random.choice(item_list)
        
    non_visited.append([curr_user, tmp_business])

In [51]:
non_visited = shuffle(non_visited)

In [52]:
non_visited[0]

[667157, 963476]

In [53]:
y_test = []
X_test = []
for i in range(len(df_test)):
    X_test.append(non_visited[i])
    y_test.append(0)
    
for idx, row in df_test.iterrows():
    X_test.append([row['user_id'], row['item_id']])
    y_test.append(1)

In [55]:
X_test, y_test = shuffle(X_test, y_test)

In [59]:
X_test[:10]

[[114890, 124553],
 [494546, 227818],
 [672080, 136860],
 [617123, 849468],
 [507231, 2822800],
 [560736, 2489222],
 [848226, 1279089],
 [460060, 131117],
 [597886, 168610],
 [827752, 648758]]

In [61]:
user_itemList[648758]

{125465}

In [58]:
y_test[:10]

[1, 1, 1, 1, 0, 0, 0, 1, 0, 0]

In [62]:
len(non_visited)

192544

In [31]:
#item_userList[2260466]

In [28]:
def calculate_jaccard(A_list, B_list):
    return float(len(set(A_list)&set(B_list))) / float(len(set(A_list) | set(B_list)))
    
    
def jaccard_score(user, item, user_itemList, item_userList):
    # get the purchase history of current user
    curr_items = user_itemList[user]
    # get the users lists who have purchase this item
    curr_users = item_userList[item]
    
    # get a list of jaccard score between targetUser and 
    # users who purchased the target items
    jaccard_list = []
    for i in curr_users:
        jaccard_list.append(calculate_jaccard(curr_items, list(user_itemList[i])))
    
    #print(jaccard_list)
    
    return max(jaccard_list)

In [26]:
len(user_itemList[47899])

4

In [29]:
jaccard_score(273551,153475,user_itemList, item_userList)

1.0

In [ ]:
# construct test set of 50% visited user-item pair and others not-visited
